### Add spark streaming kafka to Jupyter

We have a set of stock portfolios


In [ ]:
import kafka.serializer.StringDecoder

In [ ]:
import org.apache.spark._
import org.apache.spark.storage._
import org.apache.spark.streaming._
import org.apache.spark.streaming.kafka._
import com.datastax.spark.connector.streaming._
import com.datastax.spark.connector.writer.WriteConf
import scala.util.Try

### Create Case Classes for the trades and portfolios
- To make life easy

In [ ]:
case class Trade (
stock_symbol:String,
exchange:String,
trade_timestamp: String,
price: Float,
quantity: Int)

In [ ]:
case class Portfolio (
    name: String,
    stock_symbol: String,
    quantity: Int,
    price: Option[Float],
    value: Option[Float]
  )

### Roll up the portfolios every 5 seconds

In [ ]:
// The batch interval sets how we collect data for, before analyzing it in a batch
val BatchInterval = Seconds(5)

 Create a new `StreamingContext`, using the SparkContext and batch interval:

In [ ]:
val ssc = new StreamingContext(sc, BatchInterval)

 Create a Kafka stream 
 
 ### Ensure that each node defines the kafka host on every cluster node.  Get the ip address from someone knowlegeable

In [ ]:
 val directKafkaStream = KafkaUtils.createDirectStream[
     String, String, StringDecoder, StringDecoder ](
     ssc, Map("metadata.broker.list" ->"kafka:9092"), Set("Trades"))

split the strings coming in and turn it into an instance of  Trade 

In [ ]:
val trades = directKafkaStream
  .map{ case (tid, data) 
                => data.split('|') match { case Array(ss,ex,dt,p,q)
                            => Trade(ss,ex,dt,Try(p.toFloat).getOrElse(0F),Try(q.toInt).getOrElse(0))}}

create an RDD for the stock portfolios table

In [ ]:
val portfolios = sc.cassandraTable[Portfolio]("stock","portfolios").keyBy[String]("stock_symbol")

For each batch,
- You can choose to use either RDDs, SparkSQL, or DataFrames

- get the newest trade for each symbol. RDD Hint: Use a reduce for this
- join it to portfolios
- set the price for the item to the new price
- set the value for the item to the new price * quantity
- save it to the porfolios table

In [ ]:
trades.foreachRDD ( tradesRDD => tradesRDD.map( t => (t.stock_symbol, (t.trade_timestamp,t.price)))
                                         .reduceByKey( (l,r) => if (r._1 > l._1) r else l)
                                         .join(portfolios)
                                         .map{case (stock_symbol,((tt,price), port))
                                                    => port.copy(price = Some(price),
                                                                 value = Some(port.quantity * price))
                                             }
                                         .saveToCassandra("stock","portfolios")
                  )

In [ ]:
ssc.start

In [ ]:
%%cql select * from stock.portfolios limit 50

Put this in the terminal below:

```
 watch "echo \"select * from stock.portfolios where name = 'Ehtel Murakami' ;\" | cqlsh node0"
```

In [1]:
%%html <iframe src="/terminals/1" width=1000 height=400/>

### Stop the stream

In [ ]:
ssc.stop()